In [1]:
import pandas as pd
import ScraperFC as sfc
import LanusStats as ls
import math
import pickle as pkl
from mplsoccer import PyPizza, FontManager, grid
from scipy import stats

In [4]:
# fbref = ls.Fbref()
# df = fbref.get_all_player_season_stats(league='Premier League', season='2023-2024')

In [5]:
df_players = pd.read_csv('D:\Futbol\Analisis de datos\Diagramas de renidmiento\data\PL_players_23-24.csv')
df_players = df_players[df_players['Citizenship'] != '[]']
df_players.rename(columns={'Name':'Player'}, inplace=True)
df_stats = pd.read_csv('D:\Futbol\Analisis de datos\Diagramas de renidmiento\data\PL_players_noGK_stats.csv')
df_stats.drop(columns=['Unnamed: 0'], inplace=True)

In [6]:
def extract_positions(text):
    # Dividir el texto por saltos de línea y espacios en blanco
    lines = text.split('\n')
    positions = []
    for line in lines:
        # Dividir por espacios y quitar espacios en blanco
        pos = ' '.join(line.split())[2:].strip()
        # Si la línea no está vacía y no contiene solo números, añadir a la lista
        if pos and not pos[0].isdigit():
            positions.append(pos)
    return positions

In [7]:
def filter_by_position(df, position):
    return df[df['Positions'].apply(lambda positions: position in positions)]

In [8]:
df_players['Positions'] = df_players.apply(lambda row: [str(row['Position'])] if str(row['Other positions']) == 'nan' else  [str(row['Position'])] + extract_positions(str(row['Other positions'])), axis=1)

In [9]:
df_players = df_players[['Player', 'Positions']].reset_index(drop=True)

In [10]:
df = pd.merge(df_stats, df_players, on='Player')

In [11]:
df_filt = filter_by_position(df, 'Attacking Midfield')

In [13]:
with open('D:\Futbol\Analisis de datos\Diagramas de renidmiento\pkl\EPL_players_stats.pkl', 'wb') as archivo:
    pkl.dump(df, archivo, protocol=pkl.HIGHEST_PROTOCOL)

In [2]:
leages = ['Bundesliga', 'Serie A', 'La Liga', 'Ligue 1']

In [3]:
for league in leages:
    df = sfc.transfermarkt.Transfermarkt().scrape_players(year='23/24', league=league)
    df.to_csv(f'D:\Futbol\Analisis de datos\Diagramas de renidmiento\data\{league.replace(" ", "_").lower()}_players_23-24.csv')

23/24 Ligue 1 players: 100%|██████████| 1297/1297 [25:08<00:00,  1.16s/it]
